In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from Fuentes.ClassRNMulticlase import *

def test_wine_nn(data, alphas=[0.1, 0.2, 0.3], train_sizes=[0.6, 0.8], n_runs=30, FUN='sigmoid', COSTO='ECM'):
  results = []
  for alpha in alphas:
    for train_size in train_sizes:
      accs = []
      for _ in range(n_runs):
        # Split data
        X = data.drop(columns='Class')
        T = pd.get_dummies(data['Class']).astype(int).to_numpy()
        X_train, X_test, T_train, T_test = train_test_split(X, T, train_size=train_size, stratify=T)
        
        # Standardize
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train model
        rnm = RNMulticlase(alpha, 500, 1e-9, FUN=FUN, COSTO=COSTO)
        rnm.fit(X_train_scaled, T_train)
        
        # Test
        pred = rnm.predict(X_test_scaled)
        true = np.argmax(T_test, axis=1)
        acc = np.mean(pred == true)
        accs.append(acc)
      results.append({
        'alpha': alpha,
        'train_size': train_size,
        'mean_acc': np.mean(accs),
        'std_acc': np.std(accs)
      })
  return results

DATOS_DIR   = '../../Datos/'
data = pd.read_csv(DATOS_DIR + 'vinos.csv', sep=';')



In [ ]:
import numpy as np
import pandas as pd
from Fuentes.ClassRNMulticlase import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

MAX_ITER = 400
COTA = 1e-03

DATOS_DIR   = '../../Datos/'
data = pd.read_csv(DATOS_DIR + 'vinos.csv', sep=';')

T = pd.get_dummies(data["Class"]).astype(int).to_numpy()
X = data.iloc[:, 1:]

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

def run_experiments(X, T, FUN="sigmoid", COSTO="ECM", 
                    max_iter=400, cota=1e-3,
                    train_sizes=[0.6, 0.8],
                    alphas=[0.1, 0.2, 0.3],
                    n_runs=30):
    
    resultados = []
    
    for perc_for_training in train_sizes:
        for alpha in alphas:
            acc_train_runs, acc_test_runs, iters_runs = [], [], []

            for _ in range(n_runs):
                X_train, X_test, T_train, T_test = train_test_split(
                    X, T, train_size=perc_for_training, stratify=T, random_state=None
                )
                
                nnp = RNMulticlase(
                    alpha=alpha, n_iter=max_iter, cotaE=cota, FUN=FUN, COSTO=COSTO
                )
                nnp.fit(X_train, T_train)
                
                acc_train = nnp.accuracy(X_train, T_train)
                acc_test  = nnp.accuracy(X_test, T_test)
                
                acc_train_runs.append(acc_train)
                acc_test_runs.append(acc_test)
            
            resultados.append({
                "train_size": perc_for_training,
                "alpha": alpha,
                "mean_acc_train": np.mean(acc_train_runs),
                "mean_acc_test": np.mean(acc_test_runs),
            })
        
        df_temp = pd.DataFrame([r for r in resultados if r["train_size"] == perc_for_training])
        resultados.append({
            "train_size": perc_for_training,
            "alpha": "general",
            "mean_acc_train": df_temp["mean_acc_train"].mean(),
            "mean_acc_test": df_temp["mean_acc_test"].mean(),
        })
    
    return pd.DataFrame(resultados)